In [22]:
import numpy as np
import multiprocessing as mp
from xclib.data import data_utils
import scipy
import math
from tqdm import tqdm
from time import time
from sklearn.tree import DecisionTreeClassifier
import pickle
from joblib import Parallel, delayed
import itertools

In [2]:
path = "virus/ass3_parta_data"
trainX = data_utils.read_sparse_file(path+"/train_x.txt").toarray()
trainY = np.loadtxt(path+"/train_y.txt")
testX = data_utils.read_sparse_file(path+"/test_x.txt").toarray()
testY = np.loadtxt(path+"/test_y.txt")
valX = data_utils.read_sparse_file(path+"/valid_x.txt").toarray()
valY = np.loadtxt(path+"/valid_y.txt")

/home/kartik/.local/lib/python3.7/site-packages/xclib-0.96-py3.7-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [3]:
clf = DecisionTreeClassifier(criterion='entropy',random_state=0)

In [4]:
st = time()
clf.fit(trainX,trainY)
print(time()-st)

9.57831859588623


In [5]:
clf.score(trainX,trainY)

0.9127223278166674

In [6]:
class Node:
    def __init__(self,attr,splitVal,value,left,right):
        self.left = left
        self.right = right
        self.attr = attr
        self.splitVal = splitVal
        self.majority = value
        
class DecisionTree:
    def __init__(self):
        return
    
    def chooseBestAttr(self,dataX,dataY):
        pos = np.sum(dataY==1)
        neg = dataY.shape[0]-pos
        HY = -(pos/dataY.shape[0])*math.log(pos/dataY.shape[0])-(neg/dataY.shape[0])*math.log(neg/dataY.shape[0])
        entropyList=[]
        for i in (range(dataX.shape[1])):
            seq = dataX[:,i]
            splitVal = np.median(seq)
            indices0 = np.where(dataX[:,i] <= splitVal)[0]
            indices1 = np.where(dataX[:,i] > splitVal)[0]
            if(indices0.shape[0]==0 or indices1.shape[0]==0):
                entropyList.append(0)
                continue

            D0,Y0 = dataX[indices0,:],dataY[indices0]
            D1,Y1 = dataX[indices1,:],dataY[indices1]

            pos = np.sum(Y0==1)
            neg = Y0.shape[0]-pos
            if(pos==0 or neg==0):
                H0=0
            else:
                H0 = -(pos/Y0.shape[0])*math.log(pos/Y0.shape[0])-(neg/Y0.shape[0])*math.log(neg/Y0.shape[0])
            H0 = (indices0.shape[0]/dataX.shape[0])*H0
            pos = np.sum(Y1==1)
            neg = Y1.shape[0]-pos
            if(pos==0 or neg==0):
                H1=0
            else:
                H1 = -(pos/Y1.shape[0])*math.log(pos/Y1.shape[0])-(neg/Y1.shape[0])*math.log(neg/Y1.shape[0])
            H1 = (indices1.shape[0]/dataX.shape[0])*H1
            entropyList.append(HY-H0-H1)
        if(sum(entropyList)==0):
            return -1
        return entropyList.index(max(entropyList))
    
    def growTree(self,dataX,dataY):
        if(np.sum(dataY==1) == dataY.shape[0]):
            return Node(-1,-1,1,None,None)
        if(np.sum(dataY==0) == dataY.shape[0]):
            return Node(-1,-1,0,None,None)
    
        attr = self.chooseBestAttr(dataX,dataY)
        s = np.sum(dataY)
        if(s > dataY.shape[0]/2):
            val=1
        else:
            val=0
        if(attr==-1):
            return Node(-1,-1,val,None,None)

        seq = dataX[:,attr]
        splitVal = np.median(seq)
        indices0 = np.where(dataX[:,attr] <= splitVal)[0]
        indices1 = np.where(dataX[:,attr] > splitVal)[0]
        D0,Y0 = dataX[indices0,:],dataY[indices0]
        D1,Y1 = dataX[indices1,:],dataY[indices1]

        return Node(attr,splitVal,val,self.growTree(D0,Y0),self.growTree(D1,Y1))

    def fit(self,dataX,dataY):
        self.root = self.growTree(dataX,dataY)

    def findDepth(self,root):
        if(root==None):
            return 0
        return max(self.findDepth(root.left),self.findDepth(root.right))+1
    
    def findNodes(self,root):
        if(root==None):
            return 0
        l = self.findNodes(root.left)
        r = self.findNodes(root.right)
        return l+r+1

    def traverseTree(self,x,node):
        if(node.left==None and node.right==None):
            return node.majority
        if(x[node.attr]<=node.splitVal):
            return self.traverseTree(x,node.left)
        return self.traverseTree(x,node.right)

    def predict(self,dataX):
        predictions = []
        for i in dataX:
            predictions.append(self.traverseTree(i,self.root))
        predictions = np.array(predictions)
        return predictions
    
    def score(self,pred,dataY):
        check = (pred==dataY)
        return np.sum(check)/dataY.shape[0]

    def checkBinary(self,root):
        if(root.left==None and root.right==None):
            return 1
        if(root.left!=None):
            l = self.checkBinary(root.left)
        else:
            l=0
        if(root.right!=None):
            r = self.checkBinary(root.right)
        else:
            r=0
        return (l and r)

    def pruneNodes(self,root,valX,valY,acc):
        if(root.left==None and root.right==None):
            return acc
        acc1 = self.pruneNodes(root.left,valX,valY,acc)
        acc2 = self.pruneNodes(root.right,valX,valY,acc1)

        if(root.left.left==None and root.left.right==None and root.right.left==None and root.right.right==None):
            tempAtt = root.attr
            tempLeft = root.left
            tempRight = root.right
            tempSplit = root.splitVal

            root.attr = -1
            root.splitVal = -1
            root.left = None
            root.right = None

            newAcc = self.score(self.predict(valX),valY)
            if(newAcc < acc2):
                root.attr = tempAtt
                root.left = tempLeft
                root.right = tempRight
                root.splitVal = tempSplit
                return acc2
            else:
                return newAcc
                
        return acc2

    def prune(self,valX,valY):
        acc = self.score(self.predict(valX),valY)
        self.pruneNodes(self.root,valX,valY,acc)

In [19]:
clf = DecisionTree()
try:
    st = time()
    pickle_in = open("tree.pickle","rb")
    clf = pickle.load(pickle_in)
    print(time()-st)
except:
    st = time()
    clf.fit(trainX,trainY)
    print(time()-st)
    with open('tree.pickle', 'wb') as f:
        pickle.dump(clf, f)

0.09637856483459473


In [9]:
clf.findNodes(clf.root)

20045

In [10]:
predTrain = clf.predict(trainX)
trainScore = clf.score(predTrain,trainY)
trainScore

0.9047023009287161

In [11]:
predTest = clf.predict(testX)
testScore = clf.score(predTest,testY)
testScore

0.7797505910713458

In [12]:
predVal = clf.predict(valX)
valScore = clf.score(predVal,valY)
valScore

0.775913220841832

In [21]:
try:
    st = time()
    pickle_in = open("prunedTree.pickle","rb")
    clf = pickle.load(pickle_in)
    print(time()-st)
except:
    st = time()
    clf.fit(trainX,trainY)
    print(time()-st)
    with open('prunedTree.pickle', 'wb') as f:
        pickle.dump(clf, f)

In [20]:
predTrain = clf.predict(trainX)
trainScore = clf.score(predTrain,trainY)
print(trainScore)
predTest = clf.predict(testX)
testScore = clf.score(predTest,testY)
print(testScore)
predVal = clf.predict(valX)
valScore = clf.score(predVal,valY)
print(valScore)

0.9047023009287161
0.7797505910713458
0.775913220841832
